In [76]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re # re = bibliothèque de gestion des expressions régulières (regular expressions = Regex) en Python
# scanne un texte (ex : full_path) à la recherche de motifs spécifiques
# fonctions : 
# re.search() : recherche un motif dans une chaîne de caractères et renvoie un objet de correspondance si trouvé, sinon None
# re.match() : vérifie si le début d'une chaîne de caractères correspond à un motif donné
# re.findall() : trouve toutes les occurrences d'un motif dans une chaîne de caractères et les renvoie sous forme de liste
    # ex : print(len(re.findall(r'\bch\b', texte))) affiche le nombre de fois que le mot "ch" apparaît dans le texte
# re.sub() ...
# re.compile() ...
# re.split() ...
from collections import deque


In [77]:
# --- CONFIGURATION DES FILTRES ---

# Utilisation des expressions régulières (regex) pour filtrer les URLs non désirées

# Filtres pour détecter les pages commerciales inutiles
# "r" = regex = raw string = chaîne brute, interprète les backslashes littéralement
FILTRES_COMMERCIAUX = [
    r'/products?/',        # ? = dernier caractère pas obligatoire donc /products/ ou /product/
    r'/shop/',             # /shop/categorie
    r'/store/',            # /store/item
    r'/item\d+',           # \d+ : "d" équivaut à [0-9] et "+" les prend par paquet tant qu'ils se suivent, ex : item\12345
                           # \w équivaut à [a-z, A-Z, 0-9 et _] (lettres, chiffres, underscore) 
    r'/cart',              # /cart (panier)
    r'/checkout',          # /checkout (paiement)
    r'/account',           # /account (compte utilisateur)
    r'\?.*(id=|sku=|price=)'] # \? permet de garder le ? comme quand on écrit un texte entre "" 
                           #Les paramètres d'URL commerciaux commencent par ?
                           #Le .* permet de dire "n'importe quoi entre les deux"
                           #et on cherche id= ou sku= ou price= car | signifie "ou"

# Filtres pour détecter les pages techniques inutiles
FILTRES_INNUTILES = [
    r'/search',            # \search (résultats de recherche)
    r'/login',             # \login (connexion)
    r'/register',          # \register (inscription)
    r'/password',          # \password (mot de passe oublié)
    r'#',                  # évite les doublons d'url contenant plusieurs chapitres par exemple
    r'\.(pdf|jpg|png|zip)$'] # supprime les fichiers non HTML
                           # \. force à chercher le caractère "." et veut dire "n'importe quoi" 
                           # | signifie "ou" 
                           # $ signifie "à la fin de l'URL"

In [78]:

def useful_url(url):
# normaliser les liens pour pouvoir utiliser les regex dessus et ainsi trier 
# retourne True si l'URL semble être du contenu utile, False sinon
    
    parsed = urlparse(url) # urlparse est une fonction qui décompose l'URL en parties (chemin + paramètres)
    path = parsed.path.lower() # chemin de l'URL en minuscules
    query = parsed.query.lower() # paramètres de l'URL en minuscules
    full_path = path + "?" + query 
    # exemple : https://www.example.com/products/item?id=456&ref=google
    # path = /products/item
    # query = id=456&ref=google (ce qui vient après le "?")

    # 1. Vérifie si c'est un produit :
    for filtre in FILTRES_COMMERCIAUX:
        if re.search(filtre, full_path): # cherche la correspondance avec un filtre commercial dans le nouvel url "full_path"
        # pour re voir dans les imports
            return False
            
    # 2. Vérifie si c'est une page technique inutile : 
    for filtre in FILTRES_INNUTILES:
        if re.search(filtre, full_path): # cherche la correspondance avec un filtre inutile dans le nouvel url "full_path"
            return False
            
    return True

In [ ]:
def crawl_by_levels(start_url, max_levels=3):
# crawl le site niveau par niveau (BFS)
# max_levels=3 signifie : Accueil (0) -> Liens accueil (1) -> Liens de niveau 1 (2)

    domain = urlparse(start_url).netloc
    print(f"🚀 Démarrage du crawl sur : {domain}")
    print(f"🎯 Objectif : {max_levels} niveaux max")

    visited = set() #    # Set pour stocker les URLs déjà vues (évite les boucles infinies)
    texts = [] #    # Liste qui contiendra les résultats finaux (url, texte, niveau)
    
    # File d'attente pour le niveau actuel (commence avec l'URL de départ)
    current_level_queue = deque([start_url]) # # File d'attente (Queue) initiale contenant juste l'URL de départ
    # deque est optimisé pour ajouter/retirer des éléments rapidement
    
    for level in range(max_levels):
        print(f"\n{'='*60}")
        print(f"📍 NIVEAU {level} : {len(current_level_queue)} pages à explorer")
        print(f"{'='*60}")
        
        # Si la file est vide, on arrête tout (plus rien à explorer)
        if not current_level_queue:
            print("🛑 Plus aucun lien à explorer à ce niveau.")
            break
            
        # Préparation de la file pour le PROCHAIN niveau
        # Nouvelle file vide pour stocker les enfants des pages actuelles
        next_level_queue = deque()
        links_found_at_this_level = 0
        
        # On vide la file du niveau actuel
        # Tant qu'il reste des pages dans la file du niveau courant...
        while current_level_queue:
            # On prend la première URL de la file (FIFO: First In, First Out)
            url = current_level_queue.popleft()
            # Sécurité anti-boucle : si on l'a déjà vue, on passe    
            if url in visited:
                continue
            
            try:
                # Petite pause pour ne pas DDOS le site
                time.sleep(0.5)
                
                print(f"  Example ({len(visited)} total): {url}")
            
                # Téléchargement de la page (timeout 10s pour ne pas bloquer)    
                response = requests.get(url, timeout=10)
                # Si erreur (404, 500, etc.), on ignore et on passe à la suivante
                if response.status_code != 200:
                    print(f"    ⚠️ Status {response.status_code} - Ignoré")
                    continue
                
                # Analyse du HTML avec BeautifulSoup
                soup = BeautifulSoup(response.text, "html.parser")
                
                # Extraction du texte
                text_content = soup.get_text(separator=" ").strip()

                # Stockage du résultat
                texts.append({
                    "url": url, 
                    "text": text_content[:200] + "...", # On stocke les 200 premiers caractères (test)
                    "level": level})
                
                # Marquer comme visité pour ne plus jamais la refaire
                visited.add(url)
                
                # Recherche des liens pour le niveau suivant
                # On ne cherche des liens que si on n'est pas au dernier niveau demandé
                if level < max_levels - 1:
                    # Trouve toutes les balises <a href="...">
                    page_links = soup.find_all("a", href=True)
                    for a in page_links:
                        # Convertit lien relatif (/blog) en absolu (https://site.com/blog)
                        link = urljoin(url, a["href"])
                        parsed_link = urlparse(link)
                        
                        # Conditions strictes pour ajouter à la suite
                        if ("""parsed_link.netloc == domain and""" # reste sur le même domaine
                            link not in visited and                # pas déjà visité
                            link not in current_level_queue and    # pas dans la file actuelle
                            link not in next_level_queue and       # pas déjà prévu pour la suite
                            not parsed_link.fragment and           # pas d'ancre (#)
                            useful_url(link)):                     # appelle fonction filtres pour tri
                            
                            # Ajoute à la file du PROCHAIN niveau
                            next_level_queue.append(link)
                            links_found_at_this_level += 1

            except Exception as e:#en cas de crash, affiche l'erreur et on continue
                print(f"    ❌ Erreur sur {url}: {e}")
                continue

        print(f"\n✅ FIN NIVEAU {level}")
        print(f"📊 Liens trouvés pour le niveau {level+1} : {links_found_at_this_level}")
        
        # La file du prochain niveau devient la file courante pour le tour de boucle suivant

        current_level_queue = next_level_queue

    print(f"\n🎉 CRAWL TERMINÉ : {len(texts)} pages récupérées au total.")
    return texts

In [80]:
# --- EXÉCUTION ---
if __name__ == "__main__":
    start_url = "https://www.mindbodygreen.com"
    
    # Lance le crawl (3 niveaux recommandés pour test : 0, 1, 2)
    data = crawl_by_levels(start_url, max_levels=3)
    
    # Affichage des résultats
    print("\n--- 5 Premiers résultats ---")
    for d in data:
        print(f"[Niveau {d['level']}] {d['url']}")

🚀 Démarrage du crawl sur : www.mindbodygreen.com
🎯 Objectif : 3 niveaux max

📍 NIVEAU 0 : 1 pages à explorer
  Example (0 total): https://www.mindbodygreen.com

✅ FIN NIVEAU 0
📊 Liens trouvés pour le niveau 1 : 38

📍 NIVEAU 1 : 38 pages à explorer
  Example (1 total): https://www.mindbodygreen.com/accessibility
  Example (2 total): https://www.mindbodygreen.com/
  Example (3 total): https://www.mindbodygreen.com/newsletters
  Example (4 total): https://www.mindbodygreen.com/health
  Example (5 total): https://www.mindbodygreen.com/food
  Example (6 total): https://www.mindbodygreen.com/movement
  Example (7 total): https://www.mindbodygreen.com/beauty
  Example (8 total): https://www.mindbodygreen.com/articles/more-energy-with-less-caffeine-add-creatine
  Example (9 total): https://www.mindbodygreen.com/partner/california-walnuts
  Example (10 total): https://www.mindbodygreen.com/articles/want-to-elevate-your-holiday-entertaining-add-california-walnuts
  Example (11 total): https://ww

In [7]:
import requests

url = "https://www.mindbodygreen.com"

# Headers pour ressembler à un vrai navigateur (Chrome sur Windows)
HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/"
}

print(f"Tentative de connexion à {url}...")
try:
    # On met un timeout court (5s) pour ne pas attendre pour rien
    r = requests.get(url, headers=HEADERS, timeout=5)
    print(f"✅ Succès ! Status code: {r.status_code}")
    print(f"Taille de la page: {len(r.text)} caractères")
except Exception as e:
    print(f"❌ Échec : {e}")


Tentative de connexion à https://www.mindbodygreen.com...
✅ Succès ! Status code: 200
Taille de la page: 794778 caractères


In [ ]:
import re
texte = "J'ai 123@ +ch ch-ats et.j'ai 456ch_iens!"

print(re.findall(r'\d+|[a-zA-Z]+', texte))  # garde les nombres et les mots (lettres uniquement)
print(re.findall(r'\b[a-zA-Z]+(?:-[a-zA-Z]+)*\b', texte.lower())) # parfait 
print(re.findall(r'\d+\w+', texte))         # garde les nombres (et les lettres + "_" qui s'y collent)
print(re.findall(r'\S+', texte))            # ignore juste les espaces
print(re.findall(r'\s+', texte))            # garde juste les espaces
print(re.findall(r'\W+', texte))            # ignore l'alphanumérique : garde juste les caractères spéciaux
print(re.findall(r'\w+', texte))            # garde l'alphanumérique : ignore les caractères spéciaux
print(re.findall(r'\D+', texte))            # ignore juste les nombres
print(re.findall(r'\d+', texte))            # garde juste les nombres
print(re.findall(r'ch\w*', texte))          # trouve les mots commençant par "ch"
print(re.findall(r'\b\w+\b', texte))        # trouve tous les mots (séparés par des espaces ou ponctuations)
print(len(re.findall(r'\bch\b', texte)))    # compte l'occurrence du mot "ch" dans le texte

['J', 'ai', '123', 'ch', 'ch', 'ats', 'et', 'j', 'ai', '456', 'ch', 'iens']
['j', 'ai', 'ch', 'ch-ats', 'et', 'j', 'ai']
['123', '456ch_iens']
["J'ai", '123@', '+ch', 'ch-ats', "et.j'ai", '456ch_iens!']
[' ', ' ', ' ', ' ', ' ']
["'", ' ', '@ +', ' ', '-', ' ', '.', "'", ' ', '!']
['J', 'ai', '123', 'ch', 'ch', 'ats', 'et', 'j', 'ai', '456ch_iens']
["J'ai ", "@ +ch ch-ats et.j'ai ", 'ch_iens!']
['123', '456']
['ch', 'ch', 'ch_iens']
2
